In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, re, sys

from astropy.io import fits, ascii
from astropy.time import Time
from stingray import Lightcurve as LC
from stingray import CrossCorrelation
from dateutil import parser

import gbm
from gbm.data import TTE,RSP,Trigdat
from gbm.data.primitives import EventList
from gbm.binning.unbinned import bin_by_time
from gbm.plot import Lightcurve, Spectrum
from gbm.background import BackgroundFitter
from gbm.background.binned import Polynomial #working in ctime for example

# Notes
    - gbm.data.pha.PHA (pha) class is for count spectra
    - gbm.data.phaii.PHAII (phaii) class is for time history and spectra.
        can get spectra by calling this funcs: phaii.to_spectrum()
        
    - Cross corrolation is done with a ms resolution

In [ ]:
gbm.data.pha.PHA.set_properties

In [ ]:
gbm.data.phaii.PHAII.

In [9]:
def crossCorrolate(tus_F=None, kw_filename=None, binsize=None, detector_range=None, plotting=False):
    """
    cross corrolation function between FERMI and KW. Function will itself bin the photon array and return the desired time lag to shift the data into KW ref frame.
    
    Add t_shift to FERMI lc to bring it into KW ref frame.
    
    Note:   Only the bins equal to binsize will be used in KW data. 
            tus_F converted to float32 and KW data converted to float16 for memory saving. 
            --> KW data only accurate down to 1 ms
    
    input:
    --------------------------------------
    tus_F <np.array>: tus array given in seconds. Converted to float32.
    kw_filename <str>: filename.
    binsize <float>: given in seconds
    detector_range <str>: "high" or "low". High assumes BGO. Corrolates best with G2 (mid) + G3 (high). Low assumes NaI. Corrolates best with G1(low) + G2(mid)
    plotting <bool>
    
    return:
    --------------------------------------
    tlagg <float>: given in seconds. Rounded to 6 sig figures
    if value is positive the FERMI lc must be shiftet right from T0_FERMI and T0 slides to the left
    elif value is negative the FERMI lc must be shifted left og T0_FERMI and T0 slides to the right
    
    t_shift must be added to fermi tte to match the T0 to KW
    """
    

    
    #KW handling------------------------------------------------------------------------------
    if str(int(binsize*1e3)) != kw_filename[kw_filename.find('_')+1:-6]:
        return print('filename and binsize is not correct')
    
    kw_file = kw_lc_path + kw_filename
    kw_df = pd.read_csv(kw_file,sep='\s+')
    kw_bin_lo, kw_bin_hi= kw_df["Ti"].to_numpy(dtype=np.float16), kw_df["Tf"].to_numpy(dtype=np.float16)
    
    kw_dt = np.round(np.diff(kw_bin_lo),3)
    i_stop_kw = np.where(kw_dt>binsize)[0][0]
    
    kw_bin_lo, kw_bin_hi = kw_bin_lo[:i_stop_kw], kw_bin_hi[:i_stop_kw]
    
    if detector_range == "high":
        G2, G3 = kw_df["G2"].to_numpy(dtype=np.float16)[:i_stop_kw], kw_df["G3"].to_numpy(dtype=np.float16)[:i_stop_kw]
        kw_counts = G2 + G3
    elif detector_range == "low":
        G1, G2 = kw_df["G1"].to_numpy()[:i_stop_kw], kw_df["G2"].to_numpy()[:i_stop_kw]
        kw_counts = G1 + G1
    else:
        return print("detector_range not set. For NaI choose 'low' and BGO choose 'high'")
        
    kw_lc = LC(time=kw_bin_lo + binsize/2, counts=kw_counts,skip_checks=True,dt=binsize)
    check_alignment = [np.all(np.round(kw_lc.bin_lo,3) == np.round(kw_bin_lo,3)),
                       np.all(np.round(kw_lc.bin_hi,3) == np.round(kw_bin_hi,3)), 
                       np.all(np.round(kw_lc.counts,3) == np.round(kw_counts,3)) ]

    for i, bool_ in enumerate(check_alignment):
        if bool_ == False:
            print(i)
            if i == 0:
                 print("bin_lo not aligned")
            elif i == 1:
                 print("bin_hi not aligned")
            elif i == 2:
                 print("counts not aligned")
                    
    #FERMI handling-------------------------
    tus_F = np.float32(tus_F)
    kw_first_t, kw_last_t = find_nearest(tus_F,kw_bin_lo[0]), find_nearest(tus_F,kw_bin_hi[-1])
    tus_F = tus_F[kw_first_t:kw_last_t]    
    fermi_lc = LC.make_lightcurve(tus_F, dt=binsize)
    c,t = fermi_lc.counts, fermi_lc.time

    
    # Cross corrolating and shifting Fermi
    CC = CrossCorrelation(kw_lc,fermi_lc,mode="full")
    tlagg = np.round(CC.cal_timeshift(dt=binsize)[0],3)
    fermi_lc_shifted = fermi_lc.shift(tlagg) #CC.time_shift
    
    #Plotting -----------------------------
    if plotting == True:
        kw_lc.plot(title="KW lc")
        fermi_lc.plot(title="FERMI lc")
        fermi_lc_shifted.plot(title="FERMI lc shifted")
        CC.plot(labels = ['Time Lags (seconds)','Correlation'])
    
    print("tlagg (s) ", tlagg, " rounded to ms")
    return tlagg

In [8]:
def shiftTTE(tte=None,tlagg=None):
    """ Shift the fermi tte by a tshift
    input---------------
    tte <np.array>: List of photon arrival times
    tshift <float>: value to shift the tte by
    
    return----------
    shifted_tte <np.array>: tte shifted by tshift
    """
    shiftedTTE = tte + tlagg
    return shiftedTTE

In [7]:
def createNewTTE(oldTTEObj=None, shiftedTTE=None,tlagg=None binsize=None,unit=None, obj=None, grb_ra=None, grb_dec=None):
    """
    Function preserves all the channels and the whole time range
    
    Create an EventList object from lists of times, channels, 
    and the channel bounds. The list of times and channels must be the 
    same length, and the list of channel boundaries are used to map the
    PHA index number in `pha_list` to energy channels.

    input------------
    oldTTeObj <>
    shiftedTTE <np.array>
    binsize <float> : given in seconds
    unit <str>: which detector is used
    obj <str> which GRB is observed
    ra_obj <float>
    dec_obj <float>
                             
    """

    phaii = oldTTEObj.to_phaii(bin_by_time,dt=binsize)
    spectrum = phaii.to_spectrum()
    newTTEdata = EventList.from_lists(times_list = shiftedTTE, pha_list=oldTTEObj.data.pha,
                                      chan_lo=spectrum.lo_edges, chan_hi=spectrum.hi_edges)
    
    gti = [(shiftedTTE[0],shiftedTTE[-1])]
    
    #TODO: shift MET by tlagg
    newTrigtime = oldTTEOb.trigtime + tlagg
    
    newTTEObj = TTE.from_data(data=newTTEdata,gti=gti,detector=unit,object=obj, ra_obj=grb_ra, dec_obj=grb_dec)
    return newTTEObj
    